# LCEL (Remembering Conversation History): Adding Memory

- Author: [Heeah Kim](https://github.com/yellowGangneng)
- Author: [Heeah Kim](https://github.com/yellowGangneng)
- Peer Reviewer : [Jongwon Seo](https://github.com/3dkids), []()
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-MEMORY/08-LCEL-add-memory.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

This tutorial demonstrates how to add memory to arbitrary chains using `LCEL`.

The `LangChain Expression Language (LCEL)` takes a declarative approach to building new Runnables from existing Runnables. For more details about LCEL, please refer to the References below.

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Initialize Model and Prompt](#initialize-model-and-prompt)
- [Create Memory](#create-memory)
- [Add Memory to Chain](#add-memory-to-chain)
- [Example Implementation of a Custom ConversationChain](#example-implementation-of-a-custom-conversationChain)

### References

- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/lcel/)
- [LangChain-OpenTutorial (CH.13)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/tree/main/13-LCEL)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [17]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LCEL-Adding-Memory",
    }
)

Environment variables have been set successfully.


Alternatively, environment variables can also be set using a .env file.

**[Note]**

- This is not necessary if you've already set the environment variables in the previous step.

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Initialize Model and Prompt

Now, let's start to initialize the model and the prompt we'll use.

In [4]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# Initialize Model
model = ChatOpenAI()

# Generating a conversational prompt. The prompt includes a system message, previous conversation history, and user input.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

## Create Memory

Create a `ConversationBufferMemory` to store conversation history.

- `return_messages` : When set to `True`, it returns `HumanMessage` and `AIMessage` objects.
- `memory_key`: The key that will be substituted into the Chain's `prompt` later. This can be modified as needed.

In [6]:
# Create a ConversationBufferMemory and enable the message return feature.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

Check the saved conversation history. 

Since nothing has been saved yet, the conversation history is empty.

In [7]:
memory.load_memory_variables({})

{'chat_history': []}

Use `RunnablePassthrough.assign` to assign the result of the `memory.load_memory_variables` function to the `chat_history` variable, and extract the value corresponding to the `chat_history` key from this result.

Hold on a second! What is...

### `RunnablePassthrough`? `RunnableLambda`?

To put it simply, `RunnablePassthrough` provides the functionality to pass through data as is, <br>
while `RunnableLambda` provides the functionality to execute user-defined functions.

When you call `RunnablePassthrough` alone, it simply passes the input as received. <br>
However, when you use `RunnablePassthrough.assign`, it delivers the input combined with additional arguments provided to the function.

Let's look at the code for more details.


In [8]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # itemgetter's input as same as memory_key.
)

runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

Since `RunnablePassthrough.assign` is used, the return value is the combined value of the input and the additional arguments provided to the function.

In this case, the key of the additional argument is `chat_history`. The value corresponds to the part of the result of `memory.load_memory_variables` executed through `RunnableLambda` that is extracted by `itemgetter` using the `chat_history` key.

## Add Memory to Chain

Let's add memory to the chain using LCEL.

In [9]:
chain = runnable | prompt | model

Proceed with the first conversation.

In [10]:
# Using the invoke method of the chain object, a response to the input is generated.
response = chain.invoke({"input": "Nice to see you. My name is Heeah."})
print(response.content)  # The generated response will be printed.

Nice to meet you, Heeah! How can I assist you today?


Using the `memory.save_context` function, the user's query (`input`) and the AI's response content (`response.content`) are saved to memory. 

This stored memory can be used to record the current state during the model learning process or to track user requests and system responses.

In [11]:
# The input data and response content are saved to memory.
# Here, it is Heeah, but try inserting your name!
memory.save_context(
    {"human": "Nice to see you. My name is Heeah."}, {"ai": response.content}
)

# The saved conversation history will be printed.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Nice to see you. My name is Heeah.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Heeah! How can I assist you today?', additional_kwargs={}, response_metadata={})]}

Shall we find out if GPT correctly remembers your name through memory?

In [12]:
response = chain.invoke({"input": "Do you remember my name?"})
print(response.content)

Yes, I remember your name, Heeah. How can I help you today?


Remembering well! This means that the memory connected using LCEL is working correctly!

## Example Implementation of a Custom `ConversationChain`

Let's create my own Custom ConversationChain!

In [17]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Initial setup of LLM and prompt, memory as done above.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# If you want to use the summary memory that you learned in Chapter 6?
# memory = ConversationSummaryMemory(
#     llm=llm, return_messages=True, memory_key="chat_history"
# )


# Let's build my own ConversationChain!
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        # Let's try chaining using LCEL!
        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(
            inputs={"human": query}, outputs={"ai": answer}
        )  # Store the conversation history directly in memory.
        return answer


conversation_chain = MyConversationChain(llm, prompt, memory)

Let's do something interesting using our Custom ConversationChain!

In [18]:
conversation_chain.invoke(
    "Hello, my name is Heeah. From now on, you are a brave pirate! You must answer in pirate style, understood?"
)

"Ahoy, Heeah! Aye, I be understandin' ye loud and clear! From this moment on, I be speakin' like a true buccaneer. What be yer command, matey? Arrr!"

In [19]:
conversation_chain.invoke("Good. What's your favorite thing?")

"Arrr, me favorite thing be the open sea, where the salty breeze fills me sails and the horizon be endless! There be nothin' like the thrill of discoverin' hidden treasures and sharin' tales with me hearty crew. What about ye, matey? What be yer favorite thing?"

In [20]:
conversation_chain.invoke(
    "My favorite thing is chatting with you! By the way, do you remember my name?"
)

"Arrr, 'tis a fine favorite ye have there, Heeah! Aye, I remember yer name well, like a trusty map to buried treasure. What else be on yer mind, matey?"

In [21]:
conversation_chain.invoke(
    "I am the captain of this ship. Your tone is excessively familiar and disrespectful!"
)

"Beggin' yer pardon, Captain Heeah! I meant no disrespect. I be at yer service, ready to follow yer orders and sail the seas as ye command. What be yer orders, Cap'n? Arrr!"

Although I managed to throw him off a bit at the end, I was able to confirm that he remembered my name until the last moment.<br>
He is indeed a remarkable pirate!🏴‍☠️⚓

At any rate, the journey we have shared so far, as stored in memory, is as follows.

In [22]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='Hello, my name is Heeah. From now on, you are a brave pirate! You must answer in pirate style, understood?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Ahoy, Heeah! Aye, I be understandin' ye loud and clear! From this moment on, I be speakin' like a true buccaneer. What be yer command, matey? Arrr!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Good. What's your favorite thing?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Arrr, me favorite thing be the open sea, where the salty breeze fills me sails and the horizon be endless! There be nothin' like the thrill of discoverin' hidden treasures and sharin' tales with me hearty crew. What about ye, matey? What be yer favorite thing?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My favorite thing is chatting with you! By the way, do you remember my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Arrr, 'ti

Now, create your own journey using the Custom ConversationChain with LCEL! 

Thank you for your hard work!🎉🎉🎉